In [0]:
from pyspark.sql.functions import *

In [0]:
df_cases = spark.read.format("delta").load("/mnt/gold/covid_monthly_summary")
df_mobility = spark.read.format("delta").load("/mnt/gold/monthly_mobility_summary")

In [0]:
df_mobility_effect = df_mobility.alias("m") \
    .join(df_cases.alias("c"),
          on=["location", "year", "month"],
          how="inner")

In [0]:
df_mobility_effect_2 = df_mobility_effect.select(
        "location", "year", "month",
        "covid_new_cases", "covid_total_cases", "monthly_covid_case_rank", "monthly_covid_new_case_rank",
        "avg_retail", "avg_grocery", "avg_workplace", "avg_home","workplace_reduction_rank",
        "home_stay_rank"
    )

In [0]:
df_mobility_effect_3 = df_mobility_effect_2 \
    .withColumn("low_mobility_flag", when(col("avg_workplace") < -30, 1).otherwise(0)) \
    .withColumn("high_home_stay_flag", when(col("avg_home") > 10, 1).otherwise(0)) \
    .withColumn("mobility_effective_flag",
                when((col("low_mobility_flag") == 1) & (col("monthly_covid_new_case_rank") >= 10), 1).otherwise(0))

In [0]:
df_mobility_effect_3.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("/mnt/gold/mobility_effects")